In [76]:
model_name = ['llama', 'gpt', 'deepseek'][0]

import getpass
import os
from langchain_ollama import ChatOllama
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from glob import glob
import json
from num2words import num2words
import pandas as pd

In [77]:
llama = ChatOllama(
        model="llama3.2",
        temperature=0,
        num_ctx=4096,
        num_predict=2048,
        # other params...
    )
model = None
if model_name == 'llama':
    model = llama
else:
    if not os.environ.get("OPENAI_API_KEY"):
        os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
    model = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        max_tokens=None,
        timeout=None,
        max_retries=2,
        # api_key="...",  # if you prefer to pass api key in directly instaed of using env vars
        # base_url="...",
        # organization="...",
        # other params...
    )

In [78]:
# build datastores


from langchain_ollama import OllamaEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document

embeddings = OllamaEmbeddings(model="llama3.2")


In [79]:
# create small osm corpus

def extract_objects_with_geometry(data, result=None):
    """
    Recursively extract all objects with a 'geometry' attribute.

    :param data: JSON-like dictionary or list
    :param result: List to collect objects with 'geometry'
    :return: List of objects with 'geometry' attributes
    """
    if result is None:
        result = []

    if isinstance(data, dict):
        # If the current object has a 'geometry' attribute, add it to the result
        if "geometry" in data:
            result.append(data)
        
        # Recursively process all values in the dictionary
        for key, value in data.items():
            extract_objects_with_geometry(value, result)

    elif isinstance(data, list):
        # Recursively process each item in the list
        for item in data:
            extract_objects_with_geometry(item, result)

    return result


files = glob('./selected_questions/*.jsonl')
corpus = []



for path in files:
    question_type = path[path.rfind('/')+1:-6]
    with open(path, 'r') as file:
        for i in range(100):
            line = file.readline()
            question = json.loads(line)
            objs = extract_objects_with_geometry(question)
            corpus += [{k: a[k] for k in a if (a[k] is not None and k != 'geometry')} for a in objs]
            # corpus += [{k: a[k] for k in a if (a[k] is not None and k != 'geometry')} for a in [question['question_entities'][k] for k in question['question_entities']]]

print('Corpus size: ', len(corpus))

Corpus size:  166093


In [80]:
with open('./wikipedia_corpus.jsonl', 'r') as file:
    for l in file.readlines():
        if len(l.strip()):
            corpus.append(json.loads(l)['data'])
print('Corpus size: ', len(corpus))

Corpus size:  166724


In [82]:
from tqdm import tqdm 

# osm_vector_store = Chroma(collection_name="osm", embedding_function=embeddings, persist_directory="./chroma_osm_db")
osm_vector_store = Chroma(collection_name="osm", embedding_function=embeddings, persist_directory="./vectorstore")

osm_docs = []
ids = []
i = 0
for l in tqdm(corpus):
    osm_docs.append(Document(page_content=json.dumps(l)))
    ids.append(str(i))
    i+= 1
    if len(osm_docs) == 128:
        osm_vector_store.add_documents(documents=osm_docs, ids=ids)
        osm_docs = []
        ids = []
if len(osm_docs) > 0:
    osm_vector_store.add_documents(documents=osm_docs, ids=ids)
    osm_docs = []
    ids = []

100%|██████████| 166724/166724 [22:05:52<00:00,  2.10it/s]   


In [50]:
json.dumps(corpus[500])

'{"id": 25504, "poi_name": "Gibbs Gardens", "osm_id": 4519799389, "tourism": "attraction", "website": "https://www.gibbsgardens.com/", "phone": "+1 770-893-1881", "addr_state": "GA", "addr_postcode": "30107", "addr_street": "Gibbs Drive", "addr_housenumber": "1987"}'

In [83]:
emb = osm_vector_store.get(ids=["0"], include=["embeddings"])['embeddings'][0]
emb2 = embeddings.embed_query(osm_vector_store.get(ids=["0"])['documents'][0])

In [84]:
print(*zip(emb, emb2), sep='\n')

(-0.0008147391490638256, -0.00080703216)
(0.02423686906695366, 0.024231471)
(-0.030083999037742615, -0.030086635)
(-0.005084032658487558, -0.0050852937)
(-0.02107408456504345, -0.021071846)
(-0.038406334817409515, -0.038406946)
(0.009332875721156597, 0.009328611)
(0.055050019174814224, 0.05504777)
(-0.008195952512323856, -0.008187775)
(-0.006447731517255306, -0.0064494656)
(0.009482314810156822, 0.009480972)
(0.006905131042003632, 0.0068984143)
(0.0181337408721447, 0.018129759)
(0.011491990648210049, 0.011488112)
(0.011094416491687298, 0.011094764)
(0.005544369574636221, 0.0055473847)
(0.037477247416973114, 0.03748382)
(-0.00731797656044364, -0.007321516)
(0.0012815081281587481, 0.0012769089)
(-0.037767671048641205, -0.037756927)
(0.007535760290920734, 0.0075389203)
(-0.0021873177029192448, -0.0021855752)
(-0.001376081258058548, -0.001376865)
(-0.01147119328379631, -0.011470848)
(-0.03354945778846741, -0.03355315)
(-0.022686462849378586, -0.022685077)
(-0.016190430149435997, -0.0161981

In [86]:
q

'{"id": 25504, "poi_name": "Gibbs Gardens", "osm_id": 4519799389, "tourism": "attraction", "website": "https://www.gibbsgardens.com/", "phone": "+1 770-893-1881", "addr_state": "GA", "addr_postcode": "30107", "addr_street": "Gibbs Drive", "addr_housenumber": "1987"}'

In [95]:



# q = json.dumps(corpus[500])
for q in ['Los Angles', 'Pizza', 'Pizza, Los Angles']:
    print('\n\n', q)
    results = osm_vector_store.similarity_search_with_score(query=q,k=10)
    # results = osm_vector_store.similarity_search_with_score(query=q,k=10)
    # results = osm_vector_store.similarity_search_by_vector_with_relevance_scores(embedding=emb,k=10)

    for doc, score in results:
        print(f"* [SIM={score:3f}] [{doc.id}] {doc.page_content} [{doc.metadata}]")



 Los Angles
* [SIM=1.786355] [97310] {"id": 66823, "poi_name": "Skipper's Cafe", "osm_id": 11212781583, "amenity": "restaurant", "cuisine": "greek", "opening_hours": "Su 05:30-14:00; Mo-Sa 05:30-21:00", "website": "http://skipperscafe.com/", "phone": "+1-281-334-4787", "addr_state": "TX", "addr_city": "Clear Lake Shores", "addr_postcode": "77565", "addr_street": "Marina Bay Drive", "addr_housenumber": "1026", "value": 240.30239060663308} [{}]
* [SIM=1.787989] [166713] {"IUCN category IV (habitat/species management area)": "IUCN category IV (habitat/species management area)", "Location": "Jefferson County, Missouri, United States", "Nearest\u00a0city": "Pacific, MO", "Coordinates": "38\u00b025\u203200\u2033N 90\u00b042\u203223\u2033W\ufeff / \ufeff38.416581\u00b0N 90.7064\u00b0W", "Area": "1,274 acres (5.2\u00a0km2)", "Established": "2005", "Governing\u00a0body": "Missouri Department of Conservation", "Official website": "Official website"} [{}]
* [SIM=1.793527] [97471] {"id": 111279,

In [ ]:
results = osm_vector_store.similarity_search_with_score(query="qux",k=10)
for doc, score in results:
    print(f"* [SIM={score:3f}] [{doc.id}] {doc.page_content} [{doc.metadata}]")

In [ ]:
embeddings.embed_query('''"{"id": 78776, "poi_name": "The Habit Burger Grill", "osm_id": 762899568, "amenity": "fast_food", "cuisine": "burger", "wheelchair": "yes", "opening_hours": "Mo-Su 10:30-21:30", "website": "https://www.habitburger.com/locations/lake-forest/", "internet_access": "no", "phone": "+1-949-206-9110", "takeaway": "yes", "drive_through": "no", "addr_state": "CA", "addr_city": "Lake Forest", "addr_postcode": "92630", "addr_street": "El Toro Road", "addr_housenumber": "23632", "angle": 315.3741643817591}''')


[-0.011986176,
 0.016868545,
 -0.043441318,
 -0.0053491774,
 -0.016900962,
 -0.028466528,
 -0.0020697296,
 0.01558222,
 -0.0247012,
 -0.010064096,
 2.5807096e-05,
 0.012944156,
 0.039692547,
 -0.013518846,
 -0.014179126,
 0.032620266,
 0.039232127,
 -0.0059480593,
 -0.009943536,
 0.0047317618,
 0.026865523,
 0.012071038,
 0.005935781,
 -0.0039932625,
 -0.0148486495,
 -0.020908298,
 0.018438036,
 -0.018846693,
 -0.024753561,
 -0.022743488,
 0.005488499,
 0.01966795,
 0.015484767,
 0.0068696714,
 0.026605135,
 -0.017407892,
 0.0048519396,
 0.00086271804,
 -0.0043184697,
 -0.0031719208,
 -0.028043978,
 0.010386221,
 -0.013501854,
 -0.0071746013,
 -0.016210914,
 0.004143121,
 -0.030434616,
 0.009080522,
 -0.005086281,
 0.0100128455,
 0.027910389,
 0.021539867,
 -0.02670744,
 0.027646076,
 0.02889945,
 0.003668018,
 0.026461419,
 -0.009431301,
 0.004301222,
 -0.0017894646,
 0.027485235,
 0.008551331,
 0.023218062,
 0.012249946,
 0.054021012,
 0.0019303847,
 0.00629735,
 -0.0055633266,
 -0.0

In [21]:
# wiki_vector_store = Chroma(collection_name="wikipedia", embedding_function=embeddings, persist_directory="./chroma_wiki_db")
# with open('./wikipedia_corpus.jsonl', 'r') as file:
#     wiki_docs = [Document(page_content=l) for l in file.readlines()]
#     # wiki_vector_store.add_documents(documents=wiki_docs, ids=[str(i) for i in range(len(wiki_docs))])
#     osm_vector_store.add_documents(documents=wiki_docs, ids=['wiki_'+str(i) for i in range(len(wiki_docs))])

In [7]:
len(osm_vector_store.get()['ids'])

166724

In [12]:
files = glob('./selected_questions/*.jsonl')
questions = []
for path in files:
    question_type = path[path.rfind('/')+1:-6]
    with open(path, 'r') as file:
        for i in range(100):
            line = file.readline()
            question = json.loads(line)
            question['type'] = question_type
            questions.append(question)

In [66]:
system_prompt1 = str("Answer the provided user question while satisfying the following requirements:\n"
                    "1. do not include any parts of the question in the answer you must provide the answer directly.\n"
                    "2. provide only the property the user is asking for like name of an entity, its location, distance, direction.\n"
                    "3. don't provide information the user didn't ask for.\n"
                    "4. any number must be written as words and rounded to the nearest ten.\n"
                    "5. only use metric units.\n\n"
                    "The following records might be relavant to answering this question, but not necessarily:\n")


system_prompt3 = str("Given a question and a text answer, parse the text answer to json format."
                     " The location must be provided as a complete address,"
                     " any measurment must be in metric units,"
                     " and directions must be converted to azimuth angle in degress."
                     " Try to match the following schema:"
                     """
                        {
                            "name" string
                            "address": string,
                            "count": integer,
                            "distance": integer,
                            "length": integer,
                            "area": integer,
                            "azimuth_angle": integer,
                            %OTHER_ATT%
                        }
                    If a value is missing don't include it in the output, and don't write any comments.
                    All json blocks must be enclosed with ```json and ```
                     """)

# You can use the following examples as a guide:
# Example 1: What is the largest park in Tuscon, Arizona?
# ```sql
# SELECT *, ST_Area(parks.geometry::geography) AS computed_area FROM parks\nWHERE leisure = 'park'\nAND ST_Intersects(parks.geometry::geography, (SELECT geometry FROM regions WHERE wikipedia = `en:Tucson, Arizona` LIMIT 1)::geography) ORDER BY computed_area DESC LIMIT 1;
# ```
# Example 2: 
# What is the total area of all gardens in Riverside, California?
# ```sql
# SELECT SUM(ST_Area(parks.geometry::geography)) AS area FROM parks\nWHERE leisure = 'garden'\nAND ST_Intersects(parks.geometry::geography, (SELECT geometry FROM regions WHERE wikipedia = `en:Riverside, California` LIMIT 1)::geography)
# ```
# """
#)



In [44]:
import re

def flatten_if_nested(array):
    # Check if the input is a list and contains nested lists
    if isinstance(array, list) and any(isinstance(item, list) for item in array):
        flattened = []
        for item in array:
            if isinstance(item, list):
                flattened.extend(flatten_if_nested(item))
            else:
                flattened.append(item)
        return flattened
    else:
        return array  # Return the input as-is if it's not a list or doesn't contain nested lists

def extract_json_blocks(text, i):
    # Regular expression pattern to match JSON blocks
    pattern = r'```[\s]*json(.*?)```'
    pattern1 = r'\b\d+(?:_\d+)*\b'
    pattern2 = r'\b\d+(?:,\d+)*\b'
    pattern3 = r'//.*?\n'
    pattern4 = r',\s*}'
    pattern5 = r'}\s*{'
    # Find all JSON blocks
    matches = re.findall(pattern, text, re.DOTALL)
    
    # Parse each match to ensure valid JSON
    json_blocks = []
    for match in matches:
        try:
            # Remove any leading/trailing whitespace and parse as JSON
            s = match.strip()
            s = re.sub(pattern1, lambda x: x.group().replace('_', ''), s)
            s = re.sub(pattern2, lambda x: x.group().replace(',', ''), s)
            s = re.sub(pattern3, '', s)
            s = re.sub(pattern4, '}', s)
            # these are just for corrected errors in the json strings
            s = s.replace('''\\\'''', '''\'''').replace('''\\&''', '''&''').replace("""\\'""", '''\'''').replace('}\njson', '}').replace('" W', ' W').replace(""""length": 20 + 30 + 10,""", """"length": 60,""")
            if re.search(pattern5, s):
                s = re.sub(pattern5, '},\n{', s)
                s = '[\n%s\n]' % s
            convert_area = False
            if ' acres' in s:
                convert_area = True
                s = s.replace(' acres,', ',')
            json_data = json.loads(s)
            if convert_area and 'area' in json_data:
                json_data['area'] = json_data['area'] * 4046.8564224
            json_blocks.append(json_data)
        except json.JSONDecodeError as w:
            print(w)
            # If parsing fails, print an error message (can log or handle as needed)
            print(i)
            print(s)
            print("Warning: Found an invalid JSON block.") 
    return flatten_if_nested(json_blocks)

def extract_sql_blocks(text):
    # Regular expression pattern to match SQL blocks
    pattern = r'```[\s]*sql(.*?)```'
    # Find all SQL blocks
    matches = re.findall(pattern, text, re.DOTALL)
    
    sql_blocks = []
    for match in matches:
        # possibly some processing here
        sql_blocks.append(match)
    return sql_blocks



In [68]:
def remove_geo_recursively(data):
    if isinstance(data, dict):
        # Remove the 'geo_wkt' key if it exists
        if "geo_wkt" in data:
            del data["geo_wkt"]
        if "geometry" in data:
            del data["geometry"]
        # Recursively process each value in the dictionary
        for key, value in list(data.items()):
            data[key] = remove_geo_recursively(value)
    elif isinstance(data, list):
        # Recursively process each item in the list
        data = [remove_geo_recursively(item) for item in data]
    return data

In [37]:
_answers = {}
for a in answers:
    _answers[a['id']] = a

In [38]:
answers = []
for q in tqdm(questions):
    # wiki_records = ''
    # wiki_records = '\n'.join([r.page_content for r in wiki_vector_store.similarity_search(q['question'], k=)])
    if q['id'] in _answers:
        answers.append(_answers[q['id']])
        continue
    records = '\n'.join([r.page_content for r in osm_vector_store.similarity_search(q['question'], k=10)])
    # records = json.loads('[' + records + ']')
    # records = '\n'.join([json.dumps(r) for r in remove_geo_recursively(records)])
    messages = [
            SystemMessage(content= system_prompt1 + records),
            HumanMessage(content=q['question'])
        ]
    answers.append({'id':q['id'], 'content': model.invoke(messages).content})


100%|██████████| 2800/2800 [15:09<00:00,  3.08it/s] 


In [57]:
# with open('./rag_answers_%s.json' % model_name, 'w') as file:
#     file.write(json.dumps(answers, indent=2))
with open('./rag_answers_%s.json' % model_name, 'r') as file:
    answers = json.loads(file.read())

In [40]:
json_answers = []
for i in range(len(questions)):
    q = questions[i]
    if 'multihop1' in q['type']:
        sys_prompt= system_prompt3.replace('%OTHER_ATT%', '"%s": string' % q['answers'][0]['multihop_attribute'])
    else:
        sys_prompt= system_prompt3.replace('%OTHER_ATT%', '')
    a = answers[i]
    if a['id'] != q['id']:
        for j in range(len(answers)):
            if answers[j]['id'] == q['id']:
                a = answers[j]
                break
    messages = [
            SystemMessage(content=sys_prompt),
            HumanMessage(content="Question: %s\nAnswer: %s" % (q['question'], a['content']))
        ]
    json_answers.append({'id':q['id'], 'content': llama.invoke(messages).content})

In [58]:
# with open('./rag_json_answers_%s.json' % model_name, 'w') as file:
#         file.write(json.dumps(json_answers, indent=2))
with open('./rag_json_answers_%s.json' % model_name, 'r') as file:
        json_answers = json.loads(file.read())

In [45]:
a = json_answers[1500]
print(a['content'])
extract_json_blocks(a['content'], 99)

```json
{
    "name": "Fifty-Six Golf Course",
    "address": "N 45° 47' 59" W 89° 49' 01", 
    "count": null,
    "distance": 70,
    "length": null,
    "area": null,
    "azimuth_angle": 270
}
```


[{'name': 'Fifty-Six Golf Course',
  'address': "N 45° 47' 59 W 89° 49' 01",
  'count': None,
  'distance': 70,
  'length': None,
  'area': None,
  'azimuth_angle': 270}]

In [59]:
parsed_answers = []
for i in range(len(questions)):
    q = questions[i]
    a = json_answers[i]
    j = 0
    if a['id'] != q['id']:
        for j in range(len(json_answers)):
            if json_answers[j]['id'] == q['id']:
                a = json_answers[j]
                break
    # print(q['id'], a['id'], i, j)
    parsed_answers.append(extract_json_blocks(a['content'], i))

In [60]:
from geopy.geocoders import Nominatim
from pyproj import Geod

geod = Geod(ellps='WGS84')
geocoder = Nominatim(user_agent="Geocoder")

In [61]:
import importlib
import evaluate
importlib.reload(evaluate)
import numpy as np

[nltk_data] Downloading package punkt to /Users/majid/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/majid/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [49]:
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /Users/majid/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [62]:

text_evaluation = []

# evaluate text answers

for i in range(len(questions)):
    q = questions[i]
    a = answers[i]
    if a['id'] != q['id']:
        for j in range(len(answers)):
            if answers[j]['id'] == q['id']:
                a = answers[j]
                break
    text_answer = a['content']
    key = ''
    if 'multihop1' in q['type']:
        key = 'multihop_long_answer'
    elif 'name' in q['type']:        
        key = 'name'
    elif 'loc' in q['type']:
        key = 'address'
    elif 'angle' in q['type']:
        key = 'angle_description'
    elif 'area' in q['type']:
        key = 'area'
    elif 'length' in q['type']:
        key = 'length'
    elif 'count' in q['type']:
        key = 'count'
    elif 'distance' in q['type']:
        key = 'distance'
    true_answer = []
    for a in q['answers']:
        v = evaluate.get_osm_value(a, key)
        if v == None:
            continue
        if key in ['area', 'length', 'count', 'distance']:
            v = num2words(v)
        if 'area' == key:
            v += ' meters squared'
        elif key in ['length', 'distance']:
            v += ' meters'
        true_answer.append(v)
    if len(text_answer):
        true_answer = '\n'.join(true_answer)
        P, R, F1 = evaluate.evaluate_entity_names(text_answer, true_answer)
        text_evaluation.append({'attempted': True, 'P': P, 'R': R, 'F1': F1})
    else:
        text_evaluation.append({'attempted': False, 'P': 0, 'R': 0, 'F1': 0})
    

In [63]:
df = pd.DataFrame(text_evaluation)
df['type'] = [q['type'] for q in questions]
df['id'] = [q['id'] for q in questions]
df.to_csv(f'./{model_name}_rag_text_eval.csv', index=False)

In [64]:
def get_recursive(data, search_key):
    outputs = []
    if isinstance(data, dict):
        for key, value in data.items():
            if key == search_key and isinstance(value, str):  # Check for key and if the value is a string
                outputs.append(value)
            else:
                outputs.extend(get_recursive(value, search_key))  # Recurse for nested structures
    elif isinstance(data, list):
        for item in data:
            outputs.extend(get_recursive(item, search_key))  # Recurse for list elements
    return outputs


In [ ]:
# evaluate parsed_answers
parsed_evaluation = []
progress = tqdm(range(len(questions)))
def imporved_f1(new_f1, scores):
    return ('F1' not in scores) or (new_f1 > scores['F1'])

for i in progress:
    q = questions[i]
    parsed_answer = parsed_answers[i]
    scores = {'attempted': False}
    if 'multihop1' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'multihop_answer')
            if v == None:
                continue
            for p in parsed_answer:
                pred_answer = p.get(a['multihop_attribute'], None)
                if pred_answer == None or len(pred_answer) == 0:
                    continue
                P, R, F1 = evaluate.evaluate_entity_names(pred_answer, v)
                if imporved_f1(F1, scores):
                    scores = {'attempted': True, 'P': P, 'R': R, 'F1': F1}
    elif 'name' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'name')
            if v == None:
                continue
            for p in parsed_answer:
                # pred_answer = p.get('name', None)
                pred_answer = get_recursive(p, 'name')
                if pred_answer == None or len(pred_answer) == 0:
                    continue
                pred_answer = pred_answer[0]
                P, R, F1 = evaluate.evaluate_entity_names(pred_answer, v)
                if imporved_f1(F1, scores):
                    scores = { 'attempted': True, 'P': P, 'R': R, 'F1': F1}
    elif 'loc' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'address')
            loc = evaluate.get_osm_value(a, 'location')
            if v == None:
                continue
            for p in parsed_answer:
                # pred_answer = p.get('address', None)
                pred_answer = get_recursive(p, 'address')
                if pred_answer == None or len(pred_answer) == 0:
                    continue
                pred_answer = pred_answer[0]
                # if type(pred_answer) == type([]):
                #     pred_answer = ', '.join(pred_answer)
                P, R, F1 = evaluate.evaluate_entity_names(pred_answer, v)
                if imporved_f1(F1, scores):
                    scores.update({'attempted': True,'P': P, 'R': R, 'F1': F1})
                pred_loc = evaluate.get_location_by_address(geocoder, pred_answer)
                if pred_loc == None:
                    continue
                distance_error = evaluate.evaluate_location(geod, [pred_loc], [loc])[0]
                if distance_error > 5*10**5:
                    distance_error = 1.0 #float('inf')
                else:
                    distance_error /= 5*10**5
                if distance_error < scores.get('distance_error', float('inf')):
                    scores['distance_error'] = distance_error
    elif 'angle' in q['type']:
        for a in q['answers']:
            angle = evaluate.get_osm_value(a, 'angle')
            angle_desc = evaluate.get_osm_value(a, 'angle_description')
            if angle == None:
                continue
            for p in parsed_answer:
                pred_angle = p.get('azimuth_angle', None)
                try:
                    pred_angle = int(pred_angle)
                except:
                    continue
                pred_answer = evaluate.get_angle_desc(pred_angle)
                if pred_answer == None or len(pred_answer) == 0:
                    continue
                P, R, F1 = evaluate.evaluate_entity_names(pred_answer, angle_desc)
                if imporved_f1(F1, scores):
                    scores.update({'attempted': True,'P': P, 'R': R, 'F1': F1})
                angle_error = evaluate.evaluate_angle([pred_angle], [angle])[0]
                if angle_error < scores.get('angle_error', float('inf')):
                    scores['angle_error'] = angle_error
    elif 'area' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'area')
            if v == None:
                continue
            for p in parsed_answer:
                pred_v = p.get('area', None)
                if pred_v == None:
                    continue
                try:
                    pred_v = int(pred_v)
                except:
                    continue
                relative_error = evaluate.evaluate_measurement(pred_v, v)
                if relative_error < scores.get('relative_error', float('inf')):
                    scores['relative_error'] = relative_error
                    scores['attempted'] = True
    elif 'length' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'length')
            if v == None:
                continue
            for p in parsed_answer:
                pred_v = p.get('length', None)
                if pred_v == None:
                    continue
                try:
                    pred_v = int(pred_v)
                except:
                    continue
                relative_error = evaluate.evaluate_measurement(pred_v, v)
                if relative_error < scores.get('relative_error', float('inf')):
                    scores['relative_error'] = relative_error
                    scores['attempted'] = True
    elif 'count' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'count')
            if v == None:
                continue
            for p in parsed_answer:
                pred_v = p.get('count', None)
                if pred_v == None:
                    continue
                try:
                    pred_v = int(pred_v)
                except:
                    continue
                relative_error = evaluate.evaluate_measurement(pred_v, v)
                if relative_error < scores.get('relative_error', float('inf')):
                    scores['relative_error'] = relative_error
                    scores['attempted'] = True
    elif 'distance' in q['type']:
        for a in q['answers']:
            v = evaluate.get_osm_value(a, 'distance')
            if v == None:
                continue
            for p in parsed_answer:
                pred_v = p.get('distance', None)
                if pred_v == None:
                    continue
                try:
                    pred_v = int(pred_v)
                except:
                    continue
                relative_error = evaluate.evaluate_measurement(pred_v, v)
                if relative_error < scores.get('relative_error', float('inf')):
                    scores['relative_error'] = relative_error
                    scores['attempted'] = True
    parsed_evaluation.append(scores)

  0%|          | 0/2800 [00:00<?, ?it/s]

100%|██████████| 2800/2800 [00:02<00:00, 1209.84it/s]


In [66]:
df = pd.DataFrame(parsed_evaluation)
df['type'] = [q['type'] for q in questions]
df['id'] = [q['id'] for q in questions]
df.loc[df['P'].isna(), 'P'] = 0
df.loc[df['R'].isna(), 'R'] = 0
df.loc[df['F1'].isna(), 'F1'] = 0
df.loc[df['distance_error'].isna(), 'distance_error'] = 1.0
df.loc[df['angle_error'].isna(), 'angle_error'] = 1.0
df.loc[df['relative_error'].isna(), 'relative_error'] = 1.0

df.to_csv(f'./{model_name}_rag_parsed_eval.csv', index=False)